
# A Notebook to acquire the latest USGS Streamlow data 
<br /><br />

*Use this Jupyter Notebook to:* <br /> 
Download USGS streamflow<br />
Calculate daily, monthly, and annual streamflow observed data. <br /> 
Save results back to HydroShare. <br /> 
 <br /> <br /> <img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:120px;padding:20px">  
#### Civil and Environmental Engineering Department at the University of Washington 

## 1.  HydroShare Setup and Preparation

To run this notebook, we must import several libaries. These are listed in order of 1) Python standard libraries, 2) hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading and creation, and 3) the observatory_gridded_hydromet library that is downloaded with this notebook. 

In [4]:
#Python libraries available on CUAHSI JupyterHub 
import os
import numpy as np
import pandas as pd
import json
from datetime import datetime, timedelta

%matplotlib inline
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

#HydroShare Utilities
#from utilities import hydroshare
#hs = hydroshare.hydroshare()

## 2. Download USGS Daily and Instantaneous Data

In [3]:
import sys
!{sys.executable} -m pip install ulmo

     |████████████████████████████████| 92kB 4.8MB/s eta 0:00:011
     |████████████████████████████████| 890kB 12.2MB/s eta 0:00:01
     |████████████████████████████████| 153kB 25.7MB/s eta 0:00:01
     |████████████████████████████████| 829kB 23.0MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 23.5MB/s eta 0:00:01B 23.5MB/s eta 0:00:01�██████████████████▎      | 4.4MB 23.5MB/s eta 0:00:013MB 23.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/3b/00/2344469e2084fb287c2e0b57b72910309874c3245463acd6cf5e3db69324/appdirs-1.4.4-py2.py3-none-any.whl
     |████████████████████████████████| 4.3MB 26.5MB/s eta 0:00:01�██                            | 532kB 26.5MB/s eta 0:00:01�████████▌  | 4.0MB 26.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 7.0MB/s  eta 0:00:01
  Created wheel for html5lib: filename=html5lib-0.9999999-cp37-none-any.whl size=107221 sha256=a81d47658e6f7ef73826fabfe26e4a0722793a2e2e29ea373b43318c9ed12a98
  Stored in direc

In [5]:
import ulmo
import json
from ulmo.usgs import nwis

### Get metadata for a list of stations and save to JSON

In [6]:
insta_metadata=ulmo.usgs.nwis.get_sites(sites={'12186000','12189500'})
print(insta_metadata)

with open('Sauk_USGS_15min_meta.json', 'w') as f:
    f.write(json.dumps(insta_metadata))

making request for sites: http://waterservices.usgs.gov/nwis/dv/
processing data from request: https://waterservices.usgs.gov/nwis/dv/?format=waterml&sites=12189500%2C12186000
making request for sites: http://waterservices.usgs.gov/nwis/iv/
processing data from request: https://waterservices.usgs.gov/nwis/iv/?format=waterml&sites=12189500%2C12186000


{'12186000': {'code': '12186000', 'name': 'SAUK RIVER AB WHITE CHUCK RIVER NR  DARRINGTON, WA', 'network': 'NWIS', 'agency': 'USGS', 'location': {'latitude': '48.16872009', 'longitude': '-121.4706723', 'srs': 'EPSG:4326'}, 'timezone_info': {'uses_dst': True, 'dst_tz': {'abbreviation': 'PDT', 'offset': '-07:00'}, 'default_tz': {'abbreviation': 'PST', 'offset': '-08:00'}}, 'county': '53061', 'huc': '17110006', 'site_type': 'ST', 'state_code': '53'}, '12189500': {'code': '12189500', 'name': 'SAUK RIVER NEAR SAUK, WA', 'network': 'NWIS', 'agency': 'USGS', 'location': {'latitude': '48.42455859', 'longitude': '-121.5684634', 'srs': 'EPSG:4326'}, 'timezone_info': {'uses_dst': True, 'dst_tz': {'abbreviation': 'PDT', 'offset': '-07:00'}, 'default_tz': {'abbreviation': 'PST', 'offset': '-08:00'}}, 'county': '53057', 'huc': '17110006', 'site_type': 'ST', 'state_code': '53'}}


### Get 'instantaneous' or 15-min data for two storms of interest at two locations

In [8]:
#Sauk River near Sauk 2006 flood
data=ulmo.usgs.nwis.get_site_data('12189500',service='instantaneous',start='10/01/2006',end='11/30/2006')
df = pd.DataFrame(data['00060:00000']['values']).drop(['qualifiers'], axis=1).set_index('datetime')
df.value = df.value.apply(np.float)
#df.index = pd.to_datetime(df.index).to_period('min')
# mark bad data as NaN
df[df.values == -999999] = np.nan
df.to_csv('12189500_usgs_15min2006.csv')

processing data from request: https://nwis.waterservices.usgs.gov/nwis/iv/?format=waterml&site=12189500&startDT=2006-10-01T00%3A00%3A00&endDT=2006-11-30T00%3A00%3A00


In [9]:
#Sauk River near Sauk 2009 flood
data=ulmo.usgs.nwis.get_site_data('12189500',service='instantaneous',start='10/01/2009',end='11/30/2009')
df = pd.DataFrame(data['00060:00000']['values']).drop(['qualifiers'], axis=1).set_index('datetime')
df.value = df.value.apply(np.float)
#df.index = pd.to_datetime(df.index).to_period('min')
# mark bad data as NaN
df[df.values == -999999] = np.nan
df.to_csv('12189500_usgs_15min2009.csv')

processing data from request: https://nwis.waterservices.usgs.gov/nwis/iv/?format=waterml&site=12189500&startDT=2009-10-01T00%3A00%3A00&endDT=2009-11-30T00%3A00%3A00


In [10]:
#Sauk above White Chuck 2006 flood
data=ulmo.usgs.nwis.get_site_data('12186000',service='instantaneous',start='10/01/2006',end='11/30/2006')
df = pd.DataFrame(data['00060:00000']['values']).drop(['qualifiers'], axis=1).set_index('datetime')
df.value = df.value.apply(np.float)
#df.index = pd.to_datetime(df.index).to_period('min')
# mark bad data as NaN
df[df.values == -999999] = np.nan
df.to_csv('12186000_usgs_15min2006.csv')

processing data from request: https://nwis.waterservices.usgs.gov/nwis/iv/?format=waterml&site=12186000&startDT=2006-10-01T00%3A00%3A00&endDT=2006-11-30T00%3A00%3A00


In [11]:
#Sauk above White Chuck 2009 flood
data=ulmo.usgs.nwis.get_site_data('12186000',service='instantaneous',start='10/01/2009',end='11/30/2009')
df = pd.DataFrame(data['00060:00000']['values']).drop(['qualifiers'], axis=1).set_index('datetime')
df.value = df.value.apply(np.float)
#df.index = pd.to_datetime(df.index).to_period('min')
# mark bad data as NaN
df[df.values == -999999] = np.nan
df.to_csv('12186000_usgs_15min2009.csv')

processing data from request: https://nwis.waterservices.usgs.gov/nwis/iv/?format=waterml&site=12186000&startDT=2009-10-01T00%3A00%3A00&endDT=2009-11-30T00%3A00%3A00


### Learn more about the data

In [30]:
ulmo.usgs.nwis.get_site_data?

In [13]:
data.keys()

dict_keys(['00010:00000', '00060:00000', '00065:00000', '63680:00000'])

In [14]:
data['00060:00000'].keys()


dict_keys(['site', 'variable', 'values', 'methods', 'qualifiers', 'last_refresh'])

## Get Daily Streamflow Data for Sauk River near Sauk USGS (12189500)

In [103]:
import numpy as np
import pandas as pd
from ulmo.usgs import nwis

data_10=ulmo.usgs.nwis.get_site_data('12189500',service='daily',start='10/01/2000',period='11/30/2010')
data=ulmo.usgs.nwis.get_site_data('12189500',service='daily',period='all')

df = pd.DataFrame(data['00060:00003']['values']).drop(['qualifiers'], axis=1).set_index('datetime')

# convert data to a pandas dataframe
#df = pd.DataFrame(data['values']).drop(['last_checked','last_modified','qualifiers'], axis=1).set_index('datetime')
df.value = df.value.apply(np.float)
df.index = pd.to_datetime(df.index).to_period('D')


# mark bad data as NaN
df[df.values == -999999] = np.nan
df.to_csv('12189500_usgs_daily.csv')

# download and cache site data (this will take a long time the first time)


processing data from request: https://waterservices.usgs.gov/nwis/dv/?format=waterml&site=12189500&period=11%2F30%2F2010&startDT=2000-10-01
processing data from request: https://waterservices.usgs.gov/nwis/dv/?format=waterml&site=12189500&startDT=1851-01-01


In [74]:
df.shape

(34079, 1)

In [76]:
df[0:5]

,value
datetime,
1911-04-01,3380.0
1911-04-02,3110.0
1911-04-03,2850.0
1911-04-04,2590.0
1911-04-05,2340.0


In [83]:
df['Date']=df.index

In [84]:
df[0:5]

,value,Date
datetime,,
1911-04-01,3380.0,1911-04-01
1911-04-02,3110.0,1911-04-02
1911-04-03,2850.0,1911-04-03
1911-04-04,2590.0,1911-04-04
1911-04-05,2340.0,1911-04-05


In [104]:
# group the data by month, day & calculate means
#daily_groups = df.groupby((lambda Date: Date.month, lambda Date: Date.day))
#daily_groups=df.groupby(pd.Grouper(key='Date',freq='M')).mean()
daily_groups = df.resample('D').mean()
monthly_groups = df.resample('M').mean()
annual_groups = df.resample('Y').mean()
print(df.mean())
print(daily_groups.mean())
print(monthly_groups.mean())
print(annual_groups.mean())
monthly_groups.to_csv('12189500_usgs_monthly.csv')
annual_groups.to_csv('12189500_usgs_annual.csv')


value    4371.179964
dtype: float64
value    4371.179964
dtype: float64
value    4370.082071
dtype: float64
value    4365.454032
dtype: float64


In [101]:
annual_groups[0:5]

,value
datetime,
1911,4061.236364
1912,4832.605634
1913,NaN
1914,NaN
1915,NaN


In [102]:
monthly_groups[0:5]

,value
datetime,
1911-04,2685.333333
1911-05,5495.483871
1911-06,8516.333333
1911-07,5629.032258
1911-08,2362.903226


In [58]:
data?

In [59]:
data.keys()

dict_keys(['00010:00001', '00010:00002', '00010:00003', '00060:00003', '00065:00003', '63680:00008'])

In [61]:
ronda=data['00060:00003']['values']

In [67]:
np.size(ronda)

34079

In [70]:
print(data['00060:00003']['values'][0])
print(data['00060:00003']['values'][np.size(ronda)-1])


{'value': '3380', 'qualifiers': 'A', 'datetime': '1911-04-01T00:00:00'}
{'value': '4780', 'qualifiers': 'P', 'datetime': '2020-07-19T00:00:00'}


In [71]:
df

,value
datetime,
1911-04-01T00:00:00,3380
1911-04-02T00:00:00,3110
1911-04-03T00:00:00,2850
1911-04-04T00:00:00,2590
1911-04-05T00:00:00,2340
...,...
2020-07-15T00:00:00,4230
2020-07-16T00:00:00,4980
2020-07-17T00:00:00,5260


## 8. Save the results back into HydroShare
<a name="creation"></a>

Using the `hs_utils` library, the results of the Geoprocessing steps above can be saved back into HydroShare.  First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, *content files*.  In addition, we must define the type of resource that will be created, in this case *genericresource*.  

***Note:*** Make sure you save the notebook at this point, so that all notebook changes will be saved into the new HydroShare resource.

Move each file on the server within the 'files' list to an :EXISTING" HydroShare Generic Resource content folder.  Parent_resource is the destination resource ID for an existing Generic Resource. Files is a list of filepaths.

In [105]:
!ls

12186000_usgs_15min2006.csv  12189500_usgs_daily.csv
12186000_usgs_15min2009.csv  12189500_usgs_monthly.csv
12189500_usgs_15min2006.csv  Observatory_Skagit_Observed_Streamflow.ipynb
12189500_usgs_15min2009.csv  observedstreamflow.tar
12189500_usgs_annual.csv     Sauk_USGS_15min_meta.json


In [14]:
!mkdir observedstreamflow
!cp *.csv observedstreamflow
!cp *.json observedstreamflow
!tar -cvf observedstreamflow.tar observedstreamflow
!ls

observedstreamflow/
observedstreamflow/12186000_usgs_15min2006.csv
observedstreamflow/12186000_usgs_15min2009.csv
observedstreamflow/12189500_usgs_15min2006.csv
observedstreamflow/12189500_usgs_15min2009.csv
observedstreamflow/12189500_usgs_daily.csv
observedstreamflow/Sauk_USGS_15min_meta.json
observedstreamflow/ogh_meta.json
12186000_usgs_15min2006.csv	  Observatory_Skagit_Observed_Streamflow.ipynb
12186000_usgs_15min2009.csv	  observedstreamflow
12189500_usgs_15min2006.csv	  observedstreamflow.tar
12189500_usgs_15min2009.csv	  ogh_meta.json
12189500_usgs_daily.csv		  ogh.py
Observatory_Sauk_Climate.ipynb	  Sauk_USGS_15min_meta.json
Observatory_Sauk_Incubator.ipynb


In [ ]:
ThisNotebook='Observatory_Skagit_Observed_Streamflow.ipynb' #check name for consistency
observedstreamflow='observedstreamflow.tar'


files=[ThisNotebook,
       observedstreamflow]

In [ ]:
# for each file downloaded onto the server folder, move to a new HydroShare Generic Resource
title = 'Skagit Observatory download observed daily and instantaneous (15min) streamflow data.'
abstract = 'Observed streamflow from Sauk near Sauk (12189500) and Sauk above White Chuck(12186000). This dataset was generated January 16, 2018.'
keywords = ['Sauk', 'NWIS', 'USGS','streamflow'] 
rtype = 'genericresource'  

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title,
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)